In [2]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [4]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [5]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [8]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [9]:
eqn_SE4 = -P((c+e) * eqn_SE3.subs({f:c*(3*c-e)/(c+e)}) / (4*c))
sing_cubics_SE4 = P((c+e)^7*sing_cubics_SE3.subs({f:c*(3*c-e)/(c+e)}))

In [10]:
SE4 = Cubic(eqn_SE4, line, sing_cubics_SE4.factor())

In [11]:
SE4.eckardt_points

[(0, 0, 0, 1),
 (0, 1, 1, 0),
 (0, 0, 1, 0),
 (0, c^2 + c*e, c^2 + c*e, c^2 - 2*c*e + e^2)]

In [12]:
%time adm_SE4 = SE4.find_admissible_projectivities()

CPU times: user 393 ms, sys: 107 ms, total: 499 ms
Wall time: 889 ms


In [13]:
len(adm_SE4)

36

In [14]:
%time simm_SE4 = SE4.find_simmetries(adm_SE4)

CPU times: user 24.2 ms, sys: 53.4 ms, total: 77.6 ms
Wall time: 112 ms


In [15]:
len(simm_SE4)

12

In [16]:
G_SE4 = Group(simm_SE4)

In [17]:
G_SE4.is_group()

True

In [18]:
G_SE4.get_order()

12

In [19]:
G_SE4.is_abelian()

False

In [20]:
G_SE4.get_divisors_of_order()

[1, 2, 3, 4, 6, 12]

In [21]:
G_SE4.get_order_of_elements()

[1, 2, 2, 3, 2, 3, 2, 2, 2, 6, 2, 6]

In [24]:
#[G_SE4.apply_to(p) for p in SE4.eckardt_points]

In [25]:
sing_cubics_SE4.factor()

(-32768) * e * (-3*c + e) * (c + e) * (3*c + e)^2 * (-c + e)^8 * c^25

In [26]:
find_conditions_for_subfamilies(SE4, sing_cubics_SE4, adm_SE4, simm_SE4)

[(3*c^2 + e^2,)]